In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'Work/github'))
	print(os.getcwd())
except:
	pass


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

##

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.keras import BalancedBatchGenerator


only_test = False

data_file = "~/.kaggle/creditcard.csv"

raw = pd.read_csv(data_file)
if only_test: raw = raw.head(60000)
raw.describe().T

X = np.array(raw[["Amount", "Time"] + ["V%d" % i for i in range(1, 29)]]).astype(np.float32)
y = np.array(raw.Class.astype(np.int64))

scaler = StandardScaler()

X = scaler.fit_transform(X).astype(np.float32)


print("loaded %s and got %s from %s" % (data_file, str((X.shape, y.shape)), raw.shape))
raw = None

##

(x_train, x_test, y_train, y_test) = train_test_split(X, y, test_size=.33, random_state=42)


training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=1000,
                                            random_state=42)


model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(x_train.shape[1], activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 10.}


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=3)

model.evaluate(x_test,  y_test, verbose=2)


loaded ~/.kaggle/creditcard.csv and got ((284807, 30), (284807,)) from (284807, 31)
Epoch 1/3
380/380 [==============================] - 6s 15ms/step - loss: 0.3198 - accuracy: 0.7739 - aucpr: 0.8911
Epoch 2/3
380/380 [==============================] - 6s 15ms/step - loss: 0.1420 - accuracy: 0.9418 - aucpr: 0.9871
Epoch 3/3
380/380 [==============================] - 6s 15ms/step - loss: 0.1138 - accuracy: 0.9535 - aucpr: 0.9920
/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
93987/1 - 5s - loss: 0.0364 - accuracy: 0.9836 - aucpr: nan


[0.06748286134530379, 0.9835722, nan]

In [3]:
(x_train, x_test, y_train, y_test) = train_test_split(X, y, 
                                                      test_size=.33, 
                                                      random_state=42,
                                                      stratify=y)

training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=512,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(x_train.shape[1], activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 10.}

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)


Epoch 1/5
744/744 [==============================] - 10s 14ms/step - loss: 0.3306 - accuracy: 0.7723 - aucpr: 0.8573
Epoch 2/5
744/744 [==============================] - 10s 14ms/step - loss: 0.1196 - accuracy: 0.9522 - aucpr: 0.9901
Epoch 3/5
744/744 [==============================] - 11s 14ms/step - loss: 0.0911 - accuracy: 0.9633 - aucpr: 0.9949
Epoch 4/5
744/744 [==============================] - 10s 14ms/step - loss: 0.0758 - accuracy: 0.9693 - aucpr: 0.9965
Epoch 5/5
744/744 [==============================] - 10s 14ms/step - loss: 0.0640 - accuracy: 0.9751 - aucpr: 0.9975
/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
93987/1 - 5s - loss: 0.0197 - accuracy: 0.9894 - aucpr: nan


[0.03926939924316181, 0.98939216, nan]

In [4]:
y_hat = model.predict_proba(x_test)

In [5]:
y_hat

array([[2.9504299e-06],
       [0.0000000e+00],
       [2.6166439e-04],
       ...,
       [4.9212508e-06],
       [3.1488496e-04],
       [4.1898610e-07]], dtype=float32)

In [6]:
average_precision_score(y_test, y_hat)

0.713396302653385

In [7]:
training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=512,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(x_train.shape[1], activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(15, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 10.}

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=5)
model.evaluate(x_test,  y_test, verbose=4)
y_hat = model.predict_proba(x_test)
print("test score = %.3f" % average_precision_score(y_test, y_hat))


Epoch 1/5
744/744 [==============================] - 12s 16ms/step - loss: 0.3026 - accuracy: 0.7701 - aucpr: 0.8796
Epoch 2/5
744/744 [==============================] - 12s 16ms/step - loss: 0.1122 - accuracy: 0.9478 - aucpr: 0.9913
Epoch 3/5
744/744 [==============================] - 13s 17ms/step - loss: 0.0773 - accuracy: 0.9626 - aucpr: 0.9961
Epoch 4/5
744/744 [==============================] - 12s 16ms/step - loss: 0.0602 - accuracy: 0.9724 - aucpr: 0.9976
Epoch 5/5
744/744 [==============================] - 13s 17ms/step - loss: 0.0481 - accuracy: 0.9838 - aucpr: 0.9983
/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
test score = 0.679


In [8]:
training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=128,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(x_train.shape[1], activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(15, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 10.}

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=20)
model.evaluate(x_test,  y_test, verbose=4)
y_hat = model.predict_proba(x_test)
print("test score = %.3f" % average_precision_score(y_test, y_hat))


Epoch 1/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.1539 - accuracy: 0.8765 - aucpr: 0.9589
Epoch 2/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0498 - accuracy: 0.9802 - aucpr: 0.9979
Epoch 3/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0353 - accuracy: 0.9889 - aucpr: 0.9989
Epoch 4/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0321 - accuracy: 0.9900 - aucpr: 0.9991
Epoch 5/20
2976/2976 [==============================] - 48s 16ms/step - loss: 0.0302 - accuracy: 0.9911 - aucpr: 0.9991
Epoch 6/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0303 - accuracy: 0.9906 - aucpr: 0.9992
Epoch 7/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0298 - accuracy: 0.9913 - aucpr: 0.9992
Epoch 8/20
2976/2976 [==============================] - 47s 16ms/step - loss: 0.0288 - accuracy: 0.9913 - aucpr: 0.9990
Epoch 9/20
2976/2976 [==================

In [9]:
training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=128,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(x_train.shape[1], activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(15, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 1/y_train.mean()} # shall we use them?

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=7, class_weight=class_weight)
model.evaluate(x_test, y_test, verbose=4)
y_hat = model.predict_proba(x_test)
print("test score = %.3f" % average_precision_score(y_test, y_hat))


Epoch 1/7
2976/2976 [==============================] - 46s 16ms/step - loss: 7.1271 - accuracy: 0.5077 - aucpr: 0.9258
Epoch 2/7
2976/2976 [==============================] - 47s 16ms/step - loss: 2.0867 - accuracy: 0.5065 - aucpr: 0.9725
Epoch 3/7
2976/2976 [==============================] - 46s 15ms/step - loss: 1.3462 - accuracy: 0.6101 - aucpr: 0.9787
Epoch 4/7
2976/2976 [==============================] - 58s 20ms/step - loss: 0.9551 - accuracy: 0.7708 - aucpr: 0.9796
Epoch 5/7
2976/2976 [==============================] - 60s 20ms/step - loss: 0.7468 - accuracy: 0.8503 - aucpr: 0.9833
Epoch 6/7
2976/2976 [==============================] - 63s 21ms/step - loss: 0.6687 - accuracy: 0.8687 - aucpr: 0.9870
Epoch 7/7
2976/2976 [==============================] - 47s 16ms/step - loss: 0.6060 - accuracy: 0.8762 - aucpr: 0.9899
/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  reca

In [13]:
training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=128,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(x_train.shape[-1],), activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 1/y_train.mean()} # shall we use them?

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit_generator(training_generator, epochs=7)
model.evaluate(x_test, y_test, verbose=4)
y_hat = model.predict_proba(x_test)
print("test score = %.3f" % average_precision_score(y_test, y_hat))


Epoch 1/7
2976/2976 [==============================] - 59s 20ms/step - loss: 0.0223 - accuracy: 0.9928 - aucpr: 0.9989
Epoch 2/7
2976/2976 [==============================] - 61s 20ms/step - loss: 0.0065 - accuracy: 0.9987 - aucpr: 0.9997
Epoch 3/7
2976/2976 [==============================] - 61s 20ms/step - loss: 0.0046 - accuracy: 0.9991 - aucpr: 0.9997
Epoch 4/7
2976/2976 [==============================] - 59s 20ms/step - loss: 0.0058 - accuracy: 0.9991 - aucpr: 0.9996
Epoch 5/7
2976/2976 [==============================] - 55s 19ms/step - loss: 0.0069 - accuracy: 0.9991 - aucpr: 0.9995
Epoch 6/7
2976/2976 [==============================] - 56s 19ms/step - loss: 0.0059 - accuracy: 0.9992 - aucpr: 0.9996
Epoch 7/7
2976/2976 [==============================] - 56s 19ms/step - loss: 0.0037 - accuracy: 0.9994 - aucpr: 0.9997
/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  reca

In [14]:
training_generator = BalancedBatchGenerator(x_train, y_train,
                                            sampler=RandomOverSampler(sampling_strategy=1.),
                                            batch_size=2048,
                                            random_state=42)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(x_train.shape[-1],), activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

from sklearn.metrics import roc_auc_score, average_precision_score

def aucpr(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.double)

class_weight = {0: 1., 1: 1/y_train.mean()} # shall we use them?

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', aucpr])

model.fit(x_train, y_train, epochs=7)
# model.fit_generator(training_generator, epochs=7)

model.evaluate(x_test, y_test, verbose=4)
y_hat = model.predict_proba(x_test)
print("test score = %.3f" % average_precision_score(y_test, y_hat))


Train on 190820 samples
Epoch 1/7
  1696/190820 [..............................] - ETA: 1:28 - loss: 0.1613 - accuracy: 0.9752 - aucpr: nan/home/yannick/bin/anaconda3_2018.12/envs/tf/lib/python3.7/site-packages/sklearn/metrics/ranking.py:528: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
190820/190820 [==============================] - 16s 85us/sample - loss: 0.0064 - accuracy: 0.9991 - aucpr: nan
Epoch 2/7
190820/190820 [==============================] - 16s 82us/sample - loss: 0.0039 - accuracy: 0.9994 - aucpr: nan
Epoch 3/7
190820/190820 [==============================] - 17s 87us/sample - loss: 0.0039 - accuracy: 0.9994 - aucpr: nan
Epoch 4/7
190820/190820 [==============================] - 16s 85us/sample - loss: 0.0038 - accuracy: 0.9994 - aucpr: nan
Epoch 5/7
190820/190820 [==============================] - 16s 83us/sample - loss: 0.0036 - accuracy: 0.9994 - aucpr: nan
Epoch 6/7
190820/190820 [==============================] - 17s 87us/sample 